# Kaggle Competition

## House Prices: Advanced Regression Techniques 

> Data can be found at https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier)
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
import statsmodels.api as sm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
from IPython.display import display

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
def display_all(df):
    with pd.option_context('display.max_columns', 1000):
        with pd.option_context('display.max_rows', 1000):
            display(df)

In [89]:
path = "C:/Users/sastu/Downloads/House-Prices-Advanced-Regression-Techniques-master/"

In [118]:
hp = pd.read_csv(f"{path}train.csv", low_memory=False)

In [119]:
hp_holdout = pd.read_csv(f"{path}test.csv") 

In [120]:
# hp.isnull().sum() # checking for misisng values

In [121]:
# hp_holdout.isnull().sum() #found some extra columns with missing values

### Imputation - Interval variables

In [122]:
numeric = ['int32', 'int64', 'float32', 'float64']

hp_numeric = hp.select_dtypes(include=numeric)

for cols in list(hp_numeric.columns):
    hp[cols].fillna(hp[cols].median(), inplace=True)

hp_holdout_numeric = hp_holdout.select_dtypes(include=numeric)

for cols in list(hp_holdout_numeric.columns):
    hp_holdout[cols].fillna(hp[cols].median(), inplace=True)

### Imputation - Categorical variables

In [123]:
string = ['object', 'category']

hp_string = hp.select_dtypes(include=string)

for cols in list(hp_string.columns):
    hp[cols].fillna(hp[cols].mode().item(), inplace=True)

hp_holdout_string = hp_holdout.select_dtypes(include=string)

for cols in list(hp_holdout_string.columns):
    hp_holdout[cols].fillna(hp[cols].mode().item(), inplace=True)

In [124]:
hp.isnull().sum().sum() #no missing values left

0

In [125]:
hp_holdout.isnull().sum().sum() #no missing values left

0

In [126]:
hp['train'] = 1
hp_holdout['train'] = 0

In [127]:
hp_appended = hp.append(hp_holdout)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## using label encoding to change string values to numbers

In [128]:
# display_all(hp_appended.head())

In [129]:
le = LabelEncoder()
column_name = list(hp_string.columns)
hp_appended[column_name] = hp_appended[column_name].apply(lambda x: le.fit_transform(x))

In [130]:
# display_all(hp_appended.head())

### feature engineering

In [131]:
usa_gdp = pd.read_csv(f"{path}USA_GDP.csv") 
#as housing prices are affected by gdp growth, I have added gdp growth column (data obtained from online source - https://countryeconomy.com/gdp/usa?year=2007)

In [132]:
hp_appended = pd.merge(hp_appended, usa_gdp, how='left', on=['MoSold', 'YrSold'])

In [133]:
hp_appended['TotalAgeOfHouse'] = hp_appended.YrSold - hp_appended.YearBuilt
hp_appended['YearsToRemodelAfterBuilt'] = hp_appended.YearRemodAdd - hp_appended.YearBuilt
hp_appended['YearsAfterRemodelBeforeSold'] = hp_appended.YrSold - hp_appended.YearRemodAdd
hp_appended['TotalSF'] = hp_appended['TotalBsmtSF'] + hp_appended['1stFlrSF'] + hp_appended['2ndFlrSF']

In [134]:
hp_appended['QSold'] = 0 #Quarter in which the house was sold
q = []

for row in range(len(hp_appended)):
    if (hp_appended['MoSold'][row] == 1 or hp_appended['MoSold'][row] == 2 or hp_appended['MoSold'][row] == 3):
        q.append(1)
    elif (hp_appended['MoSold'][row] == 4 or hp_appended['MoSold'][row] == 5 or hp_appended['MoSold'][row] == 6):
        q.append(2)
    elif (hp_appended['MoSold'][row] == 7 or hp_appended['MoSold'][row] == 8 or hp_appended['MoSold'][row] == 9):
        q.append(3)
    else:
        q.append(4)

hp_appended['QSold'] = q

In [135]:
hp_appended.shape

(2919, 88)

### Optional - either label encoding or onehot encoding

In [136]:
# hp_appended_string = ['object', 'category']

# column_names = hp_appended.select_dtypes(include=hp_appended_string)

# for cols in list(column_names.columns):
#     dummies = pd.get_dummies(hp_appended[cols], prefix=cols)
#     hp_appended[dummies.columns] = dummies
#     hp_appended.drop(cols, inplace=True, axis=1)

In [137]:
# for column in ['MSSubClass', 'MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
#        'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle',
#        'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
#        'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'BsmtFullBath', 
#         'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 
#         'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 
#         'MoSold', 'QSold', 'SaleType', 'SaleCondition', 'Alley', 'Fence', 'MiscFeature', 'PoolQC']:
#     dummies = pd.get_dummies(hp_appended[column],prefix=column)
#     hp_appended[dummies.columns] = dummies
#     hp_appended.drop(column, inplace=True, axis=1)

In [138]:
hp_appended.shape

(2919, 386)

### separate the data appended before into train and holdout samples

In [139]:
hp_train = hp_appended.loc[hp_appended.train == 1]
hp_holdout = hp_appended.loc[hp_appended.train == 0]
hp_train.reset_index(drop=True, inplace=True)
hp_holdout.reset_index(drop=True, inplace=True)
hp_train.drop('train', axis=1, inplace=True)
hp_holdout.drop('train', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


### transforming SalePrice using log10 : eveluation metric used  => root mean square log errors

In [140]:
hp_train.SalePrice = np.log10(hp_train.SalePrice+1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [141]:
# hp_train.SalePrice.plot.hist(bins=100)

## xgboost

In [142]:
X = hp_train.drop('SalePrice', axis=1)
y = hp_train.SalePrice
train_matrix = xgb.DMatrix(data = X, label = y)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [143]:
# params = {"objective":"reg:linear", "max_depth":4}

# xgb_cv = xgb.cv(dtrain=train_matrix, nfold=4, seed=123, params=params, num_boost_round=50, metrics='rmse',
#                 early_stopping_rounds= 20, as_pandas=True)

In [144]:
# xgb_cv.tail(5)

## GridSearchCV

In [145]:
%%time

gbm_param_grid = {
    'learning_rate': [0.01, 0.1, 0.2, 0.5],
    'colsample_bytree': [0.3, 0.5, 0.7, 1],
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [2, 3, 4, 5],
    'subsample': [0.5, 0.7, 1]
}

gbm = xgb.XGBRegressor()

grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, cv=4, scoring='neg_mean_squared_error', verbose=1)

grid_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

# Result 1 ==>
# Best parameters found:  {'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100, 'subsample': 1}
# Lowest RMSE found:  0.05519594063065632
# Wall time: 2min 20s

#Result 2 ==>
# Best parameters found:  {'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.7}
# Lowest RMSE found:  0.05452722536607516
# Wall time: 1h 29min 51s

Fitting 4 folds for each of 768 candidates, totalling 3072 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

...

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

Best parameters found:  {'colsample_bytree': 0.3, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.7}
Lowest RMSE found:  0.05452722536607516
Wall time: 1h 29min 51s


In [147]:
predictions_grid = grid_mse.predict(hp_holdout.drop('SalePrice', axis=1))

In [148]:
temp = 10**predictions_grid

In [149]:
hp_holdout['SalePrice'] = temp

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [150]:
final_submission = hp_holdout[['Id', 'SalePrice']]

In [151]:
final_submission.shape

(1459, 2)

In [152]:
final_submission.to_csv(f"{path}final_submission_grid_1.csv", index=False)

## RandomSearchCV

In [146]:
%%time

gbm_param_grid = {
    'learning_rate': np.arange(0.01, 1, 0.05),
    'colsample_bytree': np.arange(0.1, 1.1, 0.1),
    'n_estimators': np.arange(10, 200, 20),
    'max_depth': [2, 3, 4, 5],
    'subsample': np.arange(0.1, 1.1, 0.1)
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor()

# Perform random search: grid_mse
randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=gbm_param_grid, cv=4, n_iter=250, verbose=1, scoring='neg_mean_squared_error')


# Fit randomized_mse to the data
randomized_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

# Result 1 ==>
# Best parameters found:  {'subsample': 1, 'n_estimators': 130, 'max_depth': 5, 'learning_rate': 0.16000000000000003, 'colsample_bytree': 0.6}
# Lowest RMSE found:  0.05632697152779537
# Wall time: 1min 17s

# Result 2 ==>
# Best parameters found:  {'subsample': 0.9, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.11, 'colsample_bytree': 0.2}
# Lowest RMSE found:  0.055923448207109906
# Wall time: 24min 8s

Fitting 4 folds for each of 250 candidates, totalling 1000 fits


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

...

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\ProgramData\Anaco

Best parameters found:  {'subsample': 0.9, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.11, 'colsample_bytree': 0.2}
Lowest RMSE found:  0.055923448207109906
Wall time: 24min 8s


In [35]:
predictions_random = randomized_mse.predict(hp_holdout.drop('SalePrice', axis=1))

In [37]:
temp = 10**predictions_random

In [38]:
hp_holdout['SalePrice'] = temp

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
final_submission = hp_holdout[['Id', 'SalePrice']]

In [40]:
final_submission.shape

(1459, 2)

In [41]:
final_submission.to_csv(f"{path}final_submission_random.csv", index=False)

## Random Forest

### create train and validation sets

In [163]:
X_train, X_valid, y_train, y_valid = train_test_split(hp_train.drop('SalePrice', axis=1), hp_train.SalePrice, 
                                                      test_size=0.3, random_state=42)

### define a function to get model statistics like rmse for train/valid and r^2 for train/valid/out of bag samples

In [164]:
import cmath as math
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

## RF - 1

In [165]:
model1 = RandomForestRegressor(n_estimators=50, n_jobs=-1, min_samples_leaf=5, oob_score=True)
%time model1.fit(X_train, y_train)
print_score(model1)

Wall time: 485 ms
[(0.0424551265124632+0j), (0.06438801645988602+0j), 0.9383476331777543, 0.8704304008079131, 0.8399603806706307]


## RF - 2

In [169]:
model2 = RandomForestRegressor(n_estimators=100, n_jobs=-1, max_features=0.5, min_samples_leaf=3, oob_score=True)
%time model2.fit(X_train, y_train)
print_score(model2)

Wall time: 587 ms
[(0.03491347902913015+0j), (0.06175164329825944+0j), 0.9583057919318946, 0.8808236552504715, 0.8491361956843932]


## final prediction

In [171]:
final_pred = model2.predict(hp_holdout.drop('SalePrice', axis=1))

In [172]:
temp = 10**final_pred

In [174]:
hp_holdout.SalePrice = temp

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [177]:
# hp_holdout.head()

In [176]:
final_submission = hp_holdout[['Id', 'SalePrice']]

In [180]:
final_submission.shape

(1459, 2)

In [181]:
final_submission.to_csv("C:/Users/sastu/Downloads/final_submission.csv", index=False)